In [1]:
!pip install datasets
!pip install --upgrade accelerate
!pip install transformers[torch]
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.8 MB/s eta 0:00:00


In [2]:
from datasets import Dataset
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import accelerate
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EncoderDecoderModel
from transformers import AutoTokenizer

2024-06-22 12:13:36.037994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 12:13:36.038133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 12:13:36.148650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def read_data(src_file, trg_file):
    src_data = open(src_file).read().strip().split('\n')

    trg_data = open(trg_file).read().strip().split('\n')

    return src_data, trg_data

In [4]:
train_src_data, train_trg_data = read_data('/kaggle/input/dataset/train.en', '/kaggle/input/dataset/train.vi')
val_src, val_trg = read_data('/kaggle/input/dataset/tst2012.en', '/kaggle/input/dataset/tst2012.vi')
test_src, test_trg = read_data('/kaggle/input/dataset/tst2013.en', '/kaggle/input/dataset/tst2013.vi')

In [5]:
#num_sentences = 10000
#train_src_data = train_src_data[:num_sentences]
#train_trg_data = train_trg_data[:num_sentences]
#num_val = 500
#val_src = val_src[:num_val]
#val_trg = val_trg[:num_val]
#num_test = 500
#test_src = test_src[:num_test]
#test_trg = test_trg[:num_test]

In [6]:
len(train_src_data)

133317

In [7]:
len(train_trg_data)

133317

In [8]:
for sample_i in range(5):
    print('English sample {}:  {}'.format(sample_i + 1, train_src_data[sample_i]))
    print('Viet Nam sample {}:  {}\n'.format(sample_i + 1, train_trg_data[sample_i]))

English sample 1:  Rachel Pike : The science behind a climate headline
Viet Nam sample 1:  Khoa học đằng sau một tiêu đề về khí hậu

English sample 2:  In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
Viet Nam sample 2:  Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .

English sample 3:  I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Viet Nam sample 3:  Tôi muốn cho các bạn biết về sự to lớn của

In [9]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [10]:
import transformers
transformers.logging.set_verbosity_error()

In [11]:
train_data = {
    'id': list(range(len(train_src_data))),
    'translation': [{'en': src, 'vi': trg} for src, trg in zip(train_src_data, train_trg_data)]
}

valid_data = {
    'id': list(range(len(val_src))),
    'translation': [{'en': src, 'vi': trg} for src, trg in zip(val_src, val_trg)]
}

test_data = {
    'id': list(range(len(test_src))),
    'translation': [{'en': src, 'vi': trg} for src, trg in zip(test_src, test_trg)]
}


In [12]:
from datasets import DatasetDict, Dataset

# Convert dictionaries to Dataset format
train_dataset = Dataset.from_dict(train_data)
valid_dataset = Dataset.from_dict(valid_data)
test_dataset = Dataset.from_dict(test_data)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

# Print DatasetDict information
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 133317
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 1553
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 1268
    })
})


In [13]:
dataset_dict['train']['translation'][:5]

[{'en': 'Rachel Pike : The science behind a climate headline',
  'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'},
 {'en': 'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .',
  'vi': 'Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .'},
 {'en': 'I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .',
  'vi': 'Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng t

In [14]:
data_train = dataset_dict['train']['translation']

data_test = dataset_dict['test']['translation']

data_val = dataset_dict['validation']['translation']

In [15]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, prefix='translate')
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [16]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["vi"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [17]:
tokenized_datasets = dataset_dict.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1553 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [19]:
import evaluate

metric = evaluate.load("sacrebleu")

In [20]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [21]:
args = Seq2SeqTrainingArguments(
    output_dir="English-Vietnamese",  # Specify the output directory where model checkpoints and logs will be saved
    eval_strategy="epoch",       # Evaluation will be done at the end of each epoch
    save_strategy="epoch",             # Save model checkpoints at the end of each epoch
    learning_rate=5e-5,                # Learning rate for the optimizer
    per_device_train_batch_size=16,    # Batch size for training per GPU/CPU device
    per_device_eval_batch_size=64,     # Batch size for evaluation per GPU/CPU device
    weight_decay=0.01,                 # Weight decay parameter for regularization
    save_total_limit=3,                # Limit the total number of saved checkpoints
    num_train_epochs=10,                # Number of training epochs
    predict_with_generate=True,        # Enable generation during evaluation
    fp16=True,                         # Enable mixed precision training (if available)
    run_name="translation_fine_tune",
)


In [22]:
import wandb
wandb.login(key="fdc7d2e2f3c7033f00cdc0f0b54d120446be006c")

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [23]:
trainer.train()

wandb: Currently logged in as: viymhg1234 (sthnew). Use `wandb login --relogin` to force relogin


{'loss': 1.6522, 'grad_norm': 3.7067642211914062, 'learning_rate': 4.9701788071522865e-05, 'epoch': 0.06000240009600384}
{'loss': 1.6011, 'grad_norm': 4.111777305603027, 'learning_rate': 4.9402376095043806e-05, 'epoch': 0.12000480019200768}
{'loss': 1.5711, 'grad_norm': 4.166563510894775, 'learning_rate': 4.910236409456378e-05, 'epoch': 0.1800072002880115}
{'loss': 1.5619, 'grad_norm': 3.669222116470337, 'learning_rate': 4.8802352094083765e-05, 'epoch': 0.24000960038401536}
{'loss': 1.5671, 'grad_norm': 3.91863751411438, 'learning_rate': 4.850234009360375e-05, 'epoch': 0.3000120004800192}
{'loss': 1.565, 'grad_norm': 3.646304130554199, 'learning_rate': 4.8202328093123725e-05, 'epoch': 0.360014400576023}
{'loss': 1.5614, 'grad_norm': 3.5289034843444824, 'learning_rate': 4.790231609264371e-05, 'epoch': 0.4200168006720269}
{'loss': 1.5591, 'grad_norm': 3.4575579166412354, 'learning_rate': 4.760230409216369e-05, 'epoch': 0.48001920076803073}
{'loss': 1.5505, 'grad_norm': 3.2661163806915283

TrainOutput(global_step=83330, training_loss=1.0850529520448395, metrics={'train_runtime': 13738.0042, 'train_samples_per_second': 97.042, 'train_steps_per_second': 6.066, 'train_loss': 1.0850529520448395, 'epoch': 10.0})

In [24]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.6979155540466309, 'eval_bleu': 30.149077976871848, 'eval_runtime': 98.2208, 'eval_samples_per_second': 15.811, 'eval_steps_per_second': 0.255, 'epoch': 10.0}


{'eval_loss': 1.6979155540466309,
 'eval_bleu': 30.149077976871848,
 'eval_runtime': 98.2208,
 'eval_samples_per_second': 15.811,
 'eval_steps_per_second': 0.255,
 'epoch': 10.0}

In [25]:
trainer.save_model()